In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/housesalesprediction/kc_house_data.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
import matplotlib.pyplot as plt
count = 1
plt.figure(figsize=(30,30))
for col in df.drop(columns = ['id','date','price']).columns:
    plt.subplot(6,3,count)
    plt.scatter(df[col],df['price'])
    plt.xlabel(col)
    plt.ylabel('price')
    count=count+1

First we look for the Categorical Variables that affect the Price of the house.
According to the above describe() table, we can figure out that there are 7 Categorical Variables namely:
<li>'bedrooms'</li><li>'bathrooms'</li><li>'floors'</li><li>'waterfront'</li><li>'view'</li><li>'condition'</li><li>'grade'

In [ ]:
import seaborn as sns
lst = ['bedrooms','bathrooms','floors','waterfront','view','condition','grade']
plt.figure(figsize=(30,30))
count = 1
for col in lst:
    plt.subplot(3,3,count)
    sns.boxplot(x=col,y='price',data = df)
    count = count + 1


Therefore we can see strong impacts of columns: 'bedrooms', 'bathrroms','wavefront','grade' and 'view'

In [ ]:
print(df.columns)

In [ ]:
df_params_selected = df.drop(columns = ['zipcode','date','yr_built','lat','long','yr_renovated','id'])
df_params_selected.columns

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df_params_selected = pd.DataFrame(ss.fit_transform(df_params_selected),columns=['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'sqft_living15', 'sqft_lot15'])

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test = tts(df_params_selected.drop(columns=['price']),df_params_selected['price'],test_size = 0.3,random_state = 1)
lr = LinearRegression()
lr.fit(x_train,y_train)
print('Training R^2 socre: ',lr.score(x_train,y_train))
print('Testing R^2 score: ',lr.score(x_test,y_test))

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
test_score=[]
train_score = []
lr1 = LinearRegression()
for i in range(1,5):
    pr = PolynomialFeatures(degree = i)
    x_train_pr = pr.fit_transform(x_train)
    x_test_pr = pr.fit_transform(x_test)
    lr1.fit(x_train_pr,y_train)
    train_score.append(lr1.score(x_train_pr,y_train))
    test_score.append(lr1.score(x_test_pr,y_test))
print(train_score)
print(test_score)

Therefore polynomial equation with degree 2 would be a good fit for predicting the House Price.

In [ ]:
from sklearn.model_selection import cross_val_score
pr = PolynomialFeatures(degree = 2)
scores = cross_val_score(LinearRegression(),pr.fit_transform(df_params_selected.drop(columns=['price'])),df_params_selected['price'],cv=4)
print(scores.mean())
print(scores)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
x_train_pr = pr.fit_transform(x_train)  # See the above cell's code for pr's degree
x_test_pr = pr.fit_transform(x_test)
params = {'alpha':[0.001,0.01,0.1,1,10,100]}
GSCV = GridSearchCV(Ridge(),params,cv=5)
GSCV.fit(x_train_pr,y_train)
yhat_test = GSCV.predict(x_test_pr)
print(GSCV.best_estimator_.score(x_test_pr,y_test))
print(GSCV.best_estimator_)

We get our best estimation at alpha = 100

In [ ]:
scores = cross_val_score(Ridge(alpha=1000),pr.fit_transform(df_params_selected.drop(columns=['price'])),df_params_selected['price'],cv=4)
print('The R^2 value for 2nd degree polynomial regularized by parameter aplha is: ',scores.mean())

In [ ]:
import seaborn as sns
rr = Ridge(alpha = 100)
rr.fit(x_train_pr,y_train)
yhat_train = rr.predict(x_train_pr)
yhat_test = rr.predict(x_test_pr)
ax1 = sns.distplot(y_train,hist = False,color = 'red',label = "Actual Train set")
ax2 = sns.distplot(yhat_train,hist = False,color = 'blue',label = "predicted Train set",ax = ax1)
print("The R^2 for the training data is:",rr.score(x_train_pr,y_train),sep=" ")

In [ ]:
ax3 = sns.distplot(y_test,hist = False,color = 'red',label = "predicted Test set")
ax4 = sns.distplot(yhat_test,hist = False,color = 'blue',label = "predicted Test set",ax = ax3)
print("The R^2 value for the Test set is:",rr.score(x_test_pr,y_test),sep = " ")

The best fit for the data would be a 2nd degree polynomial (with scaled features), and regularized by alpha = 100